# TensorFlow 2.0 alpha - Tensors and Operators
### This exercise covers (1) importing TensorFlow (2) creating/using tensors (3) GPU acceleration (4) tf.data.Dataset

In [1]:
from __future__ import absolute_import, division, print_function

## Import TensorFlow
#### Version 2.0 uses Eager Execution by default - enabling a more interactive TF Frontend

In [2]:
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Tensors
#### a Multidimensional Array, tf.Tensor Objects have a Data Type and Shape - TF Library offers many operations (tf.add, tf.matmul, etc.) that work on Tensors and native python types alike

In [3]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.square(2) + tf.square(3))      # operator overloading supported as well

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


#### tf.Tensor has a Shape and Type

In [4]:
x = tf.matmul([[1]], [[2,3]])

print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


#### Differences between tf.Tensor and NumPy array - Tensors are Immutable (unchaning), and can be backed by Accelerator Memory (GPU, TPU, etc.)

### NumPy Compatibility - Conversion
#### TF operations automatically convert ndarrays to Tensors - NumPy operations explicitly convert Tensors to ndarrays using .numpy()

In [5]:
import numpy as np

ndarray = np.ones([3, 3])

print('TensorFlow operations convert NumPy arrays to Tensors automatically')
tensor = tf.multiply(ndarray, 42)
print(tensor)
print()
print('And NumPy operations convert Tensors to NumPy arrays automatically')
print(np.add(tensor, 1))
print()
print('The .numpy() method explicitly converts a Tensor to a ndarray')
print(tensor.numpy())

TensorFlow operations convert NumPy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)

And NumPy operations convert Tensors to NumPy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]

The .numpy() method explicitly converts a Tensor to a ndarray
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


## GPU Acceleration
#### Many TF operations are accelerated when using GPU for computation - TF decides automatically whether to use CPU or GPU for an operation, copying Tensors between memory if necessary 

In [7]:
x = tf.random.uniform([3,3])

print('Is there a GPU available: '),
print(tf.test.is_gpu_available())
print()
print('Is the Tensor on GPU #0: '),
print(x.device.endswith('GPU:0'))

Is there a GPU available: 
False

Is the Tensor on GPU #0: 
False


### Device Names
#### Tensor.device provides qualified string name of the device hosting the tensor contents - Required for distributed execution of a TF program

### Explicit Device Placement
#### Placement - how individual operations are assigned on a device for execution - TF decides with no explicit guidance, or the tf.device Context Manager can be used

In [9]:
import time

def time_matmul(x) :
    start = time.time()
    for loop in range(10) :
        tf.matmul(x, x)
        
    result = time.time()-start
    
    print('10 loops: {:0.2f}ms'.format(1000*result))
    
# force CPU execution

print('On CPU:')
with tf.device('CPU:0'):
    x =tf.random.uniform([1000, 1000])
    assert x.device.endswith('CPU:0')
    time_matmul(x)

# force GPU execution, if available

if tf.test.is_gpu_available() :
    print('On GPU:')
    with tf.device('GPU:0'):
        x = tf.random.uniform([1000, 1000])
        assert x.device.ednswith('GPU:)')
        time_matmul(x)

On CPU:
10 loops: 230.56ms


## Datasets
#### tf.data.Dataset is used to build Input Pipelines from simple, re-usable pieces that will feed the model
### Create a Source - Dataset
#### Created using factory functions like Dataset.from_tensors, Dataset.from_tensor_slices, etc.

In [10]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# create a CSV file

import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f :
    f.write('''Line 1
    Line 2
    Line 3
    ''''')
ds_file = tf.data.TextLineDataset(filename)

#### Apply Transformations - like map, batch, shuffle

In [11]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

#### Iterate the .Dataset Object - to loop over records

In [12]:
print('Elements of ds_tensors:')
for x in ds_tensors :
    print(x)
print()
print('\nElements in ds_file:')
for x in ds_file :
    print(x)

Elements of ds_tensors:
tf.Tensor([4 9], shape=(2,), dtype=int32)
tf.Tensor([16 25], shape=(2,), dtype=int32)
tf.Tensor([ 1 36], shape=(2,), dtype=int32)


Elements in ds_file:
tf.Tensor([b'Line 1' b'    Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'    Line 3' b'    '], shape=(2,), dtype=string)
